In [100]:
!pip install pandas


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [101]:
import pandas as pd

#abrir o arquivo
df = pd.read_csv('votacao_secao_2022_BR.csv', sep=";", encoding='latin-1')

In [102]:
#exibir cinco primeiras linhas
df.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,NR_SECAO,CD_CARGO,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_LOCAL_VOTACAO,SQ_CANDIDATO,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO
0,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,544,ELEIÇÃO GERAL FEDERAL 2022,02/10/2022,F,...,157,1,PRESIDENTE,13,LUIZ INÁCIO LULA DA SILVA,138,1015,280001607829,CENTRO ADMINISTRATIVO FABIANO MELO (ANTIGA FAC...,"AV. GOIÁS, 556"
1,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,544,ELEIÇÃO GERAL FEDERAL 2022,02/10/2022,F,...,500,1,PRESIDENTE,13,LUIZ INÁCIO LULA DA SILVA,173,2526,280001607829,UNIÃO GAÚCHA J. SIMÕES LOPES NETO,"AV. ENGENHEIRO ILDEFONSO SIMÕES LOPES, 531"
2,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,544,ELEIÇÃO GERAL FEDERAL 2022,02/10/2022,F,...,221,1,PRESIDENTE,13,LUIZ INÁCIO LULA DA SILVA,129,1180,280001607829,UTRAMIG,"AVENIDA AFONSO PENA, 3400"
3,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,544,ELEIÇÃO GERAL FEDERAL 2022,02/10/2022,F,...,92,1,PRESIDENTE,13,LUIZ INÁCIO LULA DA SILVA,138,1023,280001607829,ESCOLA ESTADUAL GUIMARAES ROSA,"RUA ELIEZER PENA, 177"
4,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,544,ELEIÇÃO GERAL FEDERAL 2022,02/10/2022,F,...,300,1,PRESIDENTE,13,LUIZ INÁCIO LULA DA SILVA,141,1139,280001607829,COLÉGIO ESTADUAL RUBENS FARRULA,"AV. PLACIDO FIGUEIREDO JUNIOR, S/N"


In [103]:
#quem são os candidatos?
candidatos = list(df.NM_VOTAVEL.unique())

#comentar ou descomentar caso queira considerar votos em branco ou votos nulo
#candidatos.remove('VOTO BRANCO')
#candidatos.remove('VOTO NULO')

candidatos

['LUIZ INÁCIO LULA DA SILVA',
 'JOSE MARIA EYMAEL',
 'LUIZ FELIPE CHAVES D AVILA',
 'KELMON LUIS DA SILVA SOUZA',
 'VOTO BRANCO',
 'SORAYA VIEIRA THRONICKE',
 'CIRO FERREIRA GOMES',
 'SOFIA PADUA MANZANO',
 'JAIR MESSIAS BOLSONARO',
 'SIMONE NASSAR TEBET',
 'LEONARDO PÉRICLES VIEIRA ROQUE',
 'VOTO NULO',
 'VERA LUCIA PEREIRA DA SILVA SALGADO']

In [104]:
#juntar cidade e estado na coluna de cidade (pois algumas cidades tem mesmo nome em estados diferentes)
def mescla_cidade_e_estado(n):
    return n.NM_MUNICIPIO + "-" + n.SG_UF

df.NM_MUNICIPIO = df.apply(mescla_cidade_e_estado, axis=1)

In [105]:
#quantos municípios eleitorais? (lembrando que tem cidades no exterior)
len(df.NM_MUNICIPIO.unique())

5710

In [106]:
#separar os dataframes de primeiro e segundo turno
df_turno1 = df[df.NR_TURNO == 1]
df_turno2 = df[df.NR_TURNO == 2]

#remover cidades do exterior
df_turno1_brazil = df_turno1[df_turno1.SG_UF != 'ZZ']
df_turno2_brazil = df_turno2[df_turno2.SG_UF != 'ZZ']

# Qual cidade votou mais no padre Kelmon?

In [107]:
#agrupar por cidades
group_by_cities = df_turno1_brazil.groupby('NM_MUNICIPIO')

In [108]:
#montar dicionario de votos para cada candidato por cidade
from tqdm import tqdm

total_cities = 0
dict_cidades = {}
for name, group_df in tqdm(group_by_cities):
    total_votos = {}
    total_votos_na_cidade = 0
    for candidato in candidatos:
        voto_por_secao = group_df[group_df.NM_VOTAVEL == candidato].QT_VOTOS
        total_votos[candidato] = 0

        for secao, votos_na_secao in voto_por_secao.items():
            total_votos[candidato] += votos_na_secao
            total_votos_na_cidade += votos_na_secao
            
    total_votos['TOTAL'] = total_votos_na_cidade
    dict_cidades[name] = total_votos
    total_cities += 1

100%|█████████████████████████████████████████████████████████████████████████████| 5570/5570 [00:28<00:00, 194.26it/s]


In [109]:
#dicionário de votos por candidato para São Paulo
dict_cidades['SÃO PAULO-SP']

{'LUIZ INÁCIO LULA DA SILVA': 3276512,
 'JOSE MARIA EYMAEL': 1689,
 'LUIZ FELIPE CHAVES D AVILA': 67714,
 'KELMON LUIS DA SILVA SOUZA': 4281,
 'VOTO BRANCO': 152752,
 'SORAYA VIEIRA THRONICKE': 51000,
 'CIRO FERREIRA GOMES': 297973,
 'SOFIA PADUA MANZANO': 5009,
 'JAIR MESSIAS BOLSONARO': 2618593,
 'SIMONE NASSAR TEBET': 558748,
 'LEONARDO PÉRICLES VIEIRA ROQUE': 8443,
 'VOTO NULO': 294142,
 'VERA LUCIA PEREIRA DA SILVA SALGADO': 2676,
 'TOTAL': 7339532}

In [110]:
#calcular porcentagem de votos que cada candidato recebeu
def retornaCidadeEmQueCandidatoFoiMaisVotado(nome_candidato):
    dict_maior_percent = {}
    for municipio in dict_cidades:
        votos_por_candidato = dict_cidades[municipio]
        if votos_por_candidato['TOTAL'] == 0:
            continue
            
        porcentagem = votos_por_candidato[nome_candidato] / votos_por_candidato['TOTAL']
        dict_maior_percent[porcentagem] = municipio
            
    return dict_maior_percent

## Lula

In [111]:
lula_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('LUIZ INÁCIO LULA DA SILVA').items(), reverse=True)
lula_porcent[:10]

[(0.9081445192896509, 'GUARIBAS-PI'),
 (0.8965330444203684, 'FARTURA DO PIAUÍ-PI'),
 (0.8829075425790754, 'SÃO BRAZ DO PIAUÍ-PI'),
 (0.8814643290329695, 'CARNAUBEIRA DA PENHA-PE'),
 (0.8802952503209243, 'BREJO DO PIAUÍ-PI'),
 (0.878757225433526, 'TERRA NOVA-PE'),
 (0.878231456873641, 'CAMPINAS DO PIAUÍ-PI'),
 (0.8782089552238806, 'NOVA SANTA RITA-PI'),
 (0.8778135048231511, 'CAPITÃO GERVÁSIO OLIVEIRA-PI'),
 (0.8776509946830755, 'BARREIRINHA-AM')]

## Bolsonaro

In [112]:
b_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('JAIR MESSIAS BOLSONARO').items(), reverse=True)
b_porcent[:10]

[(0.8176039119804401, 'NOVA PÁDUA-RS'),
 (0.8105280917833643, 'NOVA SANTA ROSA-PR'),
 (0.7880451357121073, 'QUATRO PONTES-PR'),
 (0.7832885414121671, 'NOVO PROGRESSO-PA'),
 (0.7733918128654971, 'MERCEDES-PR'),
 (0.7611218568665378, 'NOVA BASSANO-RS'),
 (0.7595528983623603, 'ITANHANGÁ-MT'),
 (0.7594859241126071, 'GUARANTÃ DO NORTE-MT'),
 (0.756089639493342, 'SÃO JOÃO DO ITAPERIÚ-SC'),
 (0.7542830321520769, 'MARIPÁ-PR')]

## Tebet

In [113]:
t_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('SIMONE NASSAR TEBET').items(), reverse=True)
t_porcent[:10]

[(0.12310730743910467, 'IVORÁ-RS'),
 (0.10274669379450661, 'COLINAS-RS'),
 (0.10083036773428232, 'LAGOINHA-SP'),
 (0.09978947368421053, 'CAPÃO ALTO-SC'),
 (0.09572697003329633, 'TAVARES-RS'),
 (0.09056364787840405, 'PONTE PRETA-RS'),
 (0.09038638839996647, 'SÃO CAETANO DO SUL-SP'),
 (0.09, 'IMIGRANTE-RS'),
 (0.0888030888030888, 'CUNHATAÍ-SC'),
 (0.08528584817244611, 'ABDON BATISTA-SC')]

## Ciro

In [114]:
c_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('CIRO FERREIRA GOMES').items(), reverse=True)
c_porcent[:10]

[(0.17634536717105526, 'SOBRAL-CE'),
 (0.17370672889926098, 'VARJOTA-CE'),
 (0.13805734495389668, 'FORQUILHA-CE'),
 (0.11610667180129007, 'MERUOCA-CE'),
 (0.10484788907284769, 'CAMOCIM-CE'),
 (0.09692671394799054, 'ALCÂNTARAS-CE'),
 (0.09193095142513047, 'IBIAPINA-CE'),
 (0.08835182250396197, 'PALHANO-CE'),
 (0.08778846153846154, 'UBAJARA-CE'),
 (0.08728146269851862, 'GROAÍRAS-CE')]

## Felipe D'avila

In [115]:
lf_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('LUIZ FELIPE CHAVES D AVILA').items(), reverse=True)
lf_porcent[:10]

[(0.026405596434979393, 'JOINVILLE-SC'),
 (0.019563626657339697, 'BLUMENAU-SC'),
 (0.019156013305013424, 'SANTA RITA DO SAPUCAÍ-MG'),
 (0.019091847265221878, 'FERNANDES TOURINHO-MG'),
 (0.017950858778625955, 'CAMBUÍ-MG'),
 (0.017593244194229415, 'CONSOLAÇÃO-MG'),
 (0.017125035440884603, 'JARAGUÁ DO SUL-SC'),
 (0.016088055855605778, 'FLORIANÓPOLIS-SC'),
 (0.01562772830452244, 'COCAL DO SUL-SC'),
 (0.015573525007487272, 'TOCOS DO MOJI-MG')]

## Soraya

In [116]:
s_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('SORAYA VIEIRA THRONICKE').items(), reverse=True)
s_porcent[:10]

[(0.05937328202308961, 'BARRA DO CHAPÉU-SP'),
 (0.022231370934540962, 'IPORANGA-SP'),
 (0.01904761904761905, 'ABAÍRA-BA'),
 (0.018002969561989608, 'CAVALCANTE-GO'),
 (0.01796221740476928, 'PIATÃ-BA'),
 (0.017806556050182113, 'BURITINÓPOLIS-GO'),
 (0.016149068322981366, 'PAES LANDIM-PI'),
 (0.015492957746478873, 'NOVA ROMA-GO'),
 (0.015153143471252016, 'CASINHAS-PE'),
 (0.015060703857384356, 'PINTADAS-BA')]

## Kelmon

In [117]:
s_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('KELMON LUIS DA SILVA SOUZA').items(), reverse=True)
s_porcent[:10]

[(0.007356414928798002, 'LIMOEIRO DE ANADIA-AL'),
 (0.005275498241500586, 'SANTA ROSA DE LIMA-SC'),
 (0.005052824988516307, 'FIGUEIRÓPOLIS D OESTE-MT'),
 (0.004865718799368089, 'SANTA QUITÉRIA DO MARANHÃO-MA'),
 (0.004842615012106538, 'MINADOR DO NEGRÃO-AL'),
 (0.004489001945234176, 'SANTANA DO MUNDAÚ-AL'),
 (0.00412058121882455, 'ACAJUTIBA-BA'),
 (0.003970504821327283, 'NOVA BOA VISTA-RS'),
 (0.003918810289389067, 'JENIPAPO DOS VIEIRAS-MA'),
 (0.0036709134096307495, 'MILAGRES DO MARANHÃO-MA')]

## Sofia Manzano

In [118]:
s_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('SOFIA PADUA MANZANO').items(), reverse=True)
s_porcent[:10]

[(0.002306805074971165, 'DOM SILVÉRIO-MG'),
 (0.0018825301204819277, 'CABRÁLIA PAULISTA-SP'),
 (0.0016806722689075631, 'SANTO ANTÔNIO DO PLANALTO-RS'),
 (0.0016703786191536749, 'MAREMA-SC'),
 (0.001664816870144284, 'CARMÉSIA-MG'),
 (0.0016122531237404273, 'ARAPUÁ-MG'),
 (0.0015568240788790867, 'ALAGOA-MG'),
 (0.0015495867768595042, 'OSCAR BRESSANE-SP'),
 (0.001401541695865452, 'MARINÓPOLIS-SP'),
 (0.0014005602240896359, 'TUPIRATINS-TO')]

## Leo Péricles

In [119]:
s_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('LEONARDO PÉRICLES VIEIRA ROQUE').items(), reverse=True)
s_porcent[:10]

[(0.0025889967637540453, 'PIEDADE DO RIO GRANDE-MG'),
 (0.0019090279308498913, 'MAUÁ-SP'),
 (0.001728608470181504, 'COQUEIRO BAIXO-RS'),
 (0.0016083634901487736, 'JERIQUARA-SP'),
 (0.0015572281339216196, 'EQUADOR-RN'),
 (0.0014917951268025858, 'OLIVEIRA FORTES-MG'),
 (0.0014036355550614264, 'RIBEIRÃO PIRES-SP'),
 (0.001402524544179523, 'CACHOEIRA DE GOIÁS-GO'),
 (0.0013986013986013986, 'SANTA INÊS-PR'),
 (0.0013951835836286904, 'SANTO ANDRÉ-SP')]

## Vera Lucia

In [120]:
s_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('VERA LUCIA PEREIRA DA SILVA SALGADO').items(), reverse=True)
s_porcent[:10]

[(0.0016092693916961698, 'MARILAC-MG'),
 (0.001336898395721925, 'ASPÁSIA-SP'),
 (0.0011723329425556857, 'SANTA ROSA DE LIMA-SC'),
 (0.0010678056593699946, 'RUBIÁCEA-SP'),
 (0.0009962143853357243, 'ASSIS BRASIL-AC'),
 (0.0009861932938856016, 'NOVA CASTILHO-SP'),
 (0.0009523809523809524, 'SÃO JOÃO DO POLÊSINE-RS'),
 (0.0009185493985267881, 'SÃO JOÃO DEL REI-MG'),
 (0.0009153318077803204, 'SENADOR AMARAL-MG'),
 (0.0009029345372460496, 'PARDINHO-SP')]

## Eymael

In [121]:
s_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('JOSE MARIA EYMAEL').items(), reverse=True)
s_porcent[:10]

[(0.0024330900243309003, 'RELVADO-RS'),
 (0.0019450522732798443, 'ROSÁRIO DA LIMEIRA-MG'),
 (0.0016257225433526012, 'BURITI DOS MONTES-PI'),
 (0.0015943877551020409, 'VIEIRAS-MG'),
 (0.001431980906921241, 'FORQUETINHA-RS'),
 (0.0013166556945358788, 'IVORÁ-RS'),
 (0.001310615989515072, 'PRESIDENTE KUBITSCHEK-MG'),
 (0.0012127512127512127, 'CERRO GRANDE DO SUL-RS'),
 (0.0012121212121212121, 'UBIRETAMA-RS'),
 (0.0011785503830288745, 'POÇO DAS ANTAS-RS')]

## Voto nulo

In [122]:
s_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('VOTO NULO').items(), reverse=True)
s_porcent[:10]

[(0.08544410672070246, 'CONGONHAS DO NORTE-MG'),
 (0.08145363408521303, 'FRUTA DE LEITE-MG'),
 (0.0798626019750966, 'SERRA AZUL DE MINAS-MG'),
 (0.07514231499051234, 'LADAINHA-MG'),
 (0.06863661866467177, 'NATUBA-PB'),
 (0.06822798422750627, 'TAPIRAMUTÁ-BA'),
 (0.06816779765576804, 'BANDEIRA-MG'),
 (0.06776145203111496, 'SETUBINHA-MG'),
 (0.06649282920469361, 'SANTA HELENA DE MINAS-MG'),
 (0.06614253973679712, 'SÃO BENEDITO DO SUL-PE')]

## Voto branco

In [123]:
s_porcent = sorted(retornaCidadeEmQueCandidatoFoiMaisVotado('VOTO BRANCO').items(), reverse=True)
s_porcent[:10]

[(0.0537828617298816, 'DESTERRO DE ENTRE RIOS-MG'),
 (0.04699496955255494, 'GUAREÍ-SP'),
 (0.04507422402159244, 'CAPELA NOVA-MG'),
 (0.04373650107991361, 'ERVÁLIA-MG'),
 (0.04228953579858379, 'SANTANA DA BOA VISTA-RS'),
 (0.039312567132116005, 'CANDEIAS-MG'),
 (0.039243498817966904, 'CESÁRIO LANGE-SP'),
 (0.039217014359252234, 'IGARAÇU DO TIETÊ-SP'),
 (0.03908462671349269, 'ITAGUARA-MG'),
 (0.03897550111358575, 'SÃO JOÃO DA URTIGA-RS')]

# Migração de votos entre Lula e Bolsonaro

In [141]:
group_by_cities_1t = df_turno1.groupby('NM_MUNICIPIO') #agrupando por cidades
group_by_cities_2t = df_turno2.groupby('NM_MUNICIPIO') #agrupando por cidades

In [130]:
#montagem de dicionario de votos para cada candidato por cidade - 1 turno
from tqdm import tqdm

total_cities = 0
dict_cidades_1t = {}
for name, group_df in tqdm(group_by_cities_1t):
    total_votos = {}
    total_votos_na_cidade = 0
    for candidato in candidatos:
        voto_por_secao = group_df[group_df.NM_VOTAVEL == candidato].QT_VOTOS
        total_votos[candidato] = 0

        for secao, votos_na_secao in voto_por_secao.items():
            total_votos[candidato] += votos_na_secao
            total_votos_na_cidade += votos_na_secao
            
    total_votos['TOTAL'] = total_votos_na_cidade
    dict_cidades_1t[name] = total_votos
    total_cities += 1

100%|█████████████████████████████████████████████████████████████████████████████| 5570/5570 [00:28<00:00, 195.65it/s]


In [142]:
#montagem de dicionario de votos para cada candidato por cidade - 2 turno
from tqdm import tqdm

total_cities = 0
dict_cidades_2t = {}
for name, group_df in tqdm(group_by_cities_2t):
    total_votos = {}
    total_votos_na_cidade = 0
    for candidato in candidatos:
        voto_por_secao = group_df[group_df.NM_VOTAVEL == candidato].QT_VOTOS
        total_votos[candidato] = 0

        for secao, votos_na_secao in voto_por_secao.items():
            total_votos[candidato] += votos_na_secao
            total_votos_na_cidade += votos_na_secao
            
    total_votos['TOTAL'] = total_votos_na_cidade
    dict_cidades_2t[name] = total_votos
    total_cities += 1

100%|█████████████████████████████████████████████████████████████████████████████| 5710/5710 [00:22<00:00, 248.68it/s]


In [132]:
import math

#ranking de cidades que mais estavam com o lula e que migraram pro bolsonaro
dict_diferencas = {}
for index, municipio in enumerate(dict_cidades_1t):
    pcent_l_1t = dict_cidades_1t[municipio]['LUIZ INÁCIO LULA DA SILVA'] / dict_cidades_1t[municipio]['TOTAL']
    pcent_b_1t = dict_cidades_1t[municipio]['JAIR MESSIAS BOLSONARO'] / dict_cidades_1t[municipio]['TOTAL']

    pcent_l_2t = dict_cidades_2t[municipio]['LUIZ INÁCIO LULA DA SILVA'] / dict_cidades_2t[municipio]['TOTAL']
    pcent_b_2t = dict_cidades_2t[municipio]['JAIR MESSIAS BOLSONARO'] / dict_cidades_2t[municipio]['TOTAL']

    votos_1t = (pcent_l_1t, pcent_b_1t)
    votos_2t = (pcent_l_2t, pcent_b_2t)
    
    diferenca = math.sqrt((pcent_l_1t - pcent_l_2t) ** 2 + (pcent_b_1t - pcent_b_2t) ** 2)

    dict_diferencas[diferenca] = municipio
    
    if municipio == 'SÃO PAULO-SP':
        print(municipio)
        print(votos_1t, votos_2t)
        print(diferenca)

SÃO PAULO-SP
(0.4464197444741708, 0.3567792878347012) (0.49889570495558677, 0.43291241438695277)
0.09246609858249824


In [152]:
sorted(dict_diferencas.items(), reverse=True)

[(0.14707009528924198, 'GRUPIARA-MG'),
 (0.14107756282659364, 'SOBRAL-CE'),
 (0.12914668528233042, 'VARJOTA-CE'),
 (0.12613037886506206, 'MIGUEL LEÃO-PI'),
 (0.12507219097216402, 'CONCEIÇÃO DAS PEDRAS-MG'),
 (0.12169366356021333, 'IMIGRANTE-RS'),
 (0.11928140366552943, 'NOVA BOA VISTA-RS'),
 (0.11848567727952485, 'CAPÃO ALTO-SC'),
 (0.11702171516251904, 'IVORÁ-RS'),
 (0.10859365404722232, 'NATÉRCIA-MG'),
 (0.10823844163397925, 'SÃO TIAGO-MG'),
 (0.1080371390689385, 'IPORÃ DO OESTE-SC'),
 (0.10676246202396258, 'POÇO DAS ANTAS-RS'),
 (0.10595907765644874, 'ALTO FELIZ-RS'),
 (0.10567312285879553, 'CÓRREGO FUNDO-MG'),
 (0.10379231355429397, 'TREVISO-SC'),
 (0.10358562893302116, 'CLÁUDIO-MG'),
 (0.10350957587189856, 'ITAGUARA-MG'),
 (0.10343076410679596, 'COLINAS-RS'),
 (0.102543211267945, 'BROCHIER-RS'),
 (0.10153802128516193, 'TURUÇU-RS'),
 (0.10123144767066297, 'CUNHATAÍ-SC'),
 (0.10087915149729387, 'RIACHO DA CRUZ-RN'),
 (0.10052993669156998, 'SÃO VENDELINO-RS'),
 (0.10042804760945556, 

# Seções (urnas) com 100% dos votos em Lula ou Bolsonaro

In [143]:
#vetorização de colunas do dataframe
l_nm_municipio = df_turno2.NM_MUNICIPIO.to_numpy()
l_nr_zona = df_turno2.NR_ZONA.to_numpy()
l_nr_secao = df_turno2.NR_SECAO.to_numpy()
l_nm_votavel = df_turno2.NM_VOTAVEL.to_numpy()
l_nm_localvotacao = df_turno2.NM_LOCAL_VOTACAO.to_numpy()
l_votos = df_turno2.QT_VOTOS.to_numpy()

In [154]:
#construção de dicionário de votos por zona e seção
index = 0
dict_turno2_municipio_zona_secao = {}
for cidade, zona, secao, candidato, local, votos in zip(l_nm_municipio, l_nr_zona, l_nr_secao, \
                                                 l_nm_votavel, l_nm_localvotacao, l_votos):
    if candidato == 'VOTO BRANCO' or candidato == 'VOTO NULO':
        continue

    if (cidade, zona, secao) not in dict_turno2_municipio_zona_secao:
        dict_turno2_municipio_zona_secao[(cidade, zona, secao)] = []
        
    dict_turno2_municipio_zona_secao[(cidade, zona, secao)].append((candidato, votos))

In [155]:
dict_turno2_municipio_zona_secao[('SÃO PAULO-SP', 1, 20)]

[('JAIR MESSIAS BOLSONARO', 94), ('LUIZ INÁCIO LULA DA SILVA', 127)]

In [156]:
#mostrar urnas que bolsonaro teve zero votos
total_urnas_sem_bolsonaro = 0

soma_votos_lula_urnas100porcent = 0
for index, chave_votos in enumerate(dict_turno2_municipio_zona_secao):   
    if len(dict_turno2_municipio_zona_secao[chave_votos]) < 4: #chance de não terem votado em L nem B
        achou_bolsonaro = False
        for t in dict_turno2_municipio_zona_secao[chave_votos]:
            if 'JAIR MESSIAS BOLSONARO' in t:
                achou_bolsonaro = True
        
        #não teve votos pro bolsonaro na urna
        if achou_bolsonaro == False:
            total_urnas_sem_bolsonaro += 1
            print(chave_votos)
            print(dict_turno2_municipio_zona_secao[chave_votos])
            print("\n")
            
            #somar votos do lula
            for t in dict_turno2_municipio_zona_secao[chave_votos]:
                if 'LUIZ INÁCIO LULA DA SILVA' in t:
                    soma_votos_lula_urnas100porcent += t[1]            

total_urnas_sem_bolsonaro

('MARANHÃOZINHO-MA', 101, 78)
[('LUIZ INÁCIO LULA DA SILVA', 307)]


('ATALAIA DO NORTE-AM', 42, 28)
[('LUIZ INÁCIO LULA DA SILVA', 127)]


('PENALVA-MA', 45, 63)
[('LUIZ INÁCIO LULA DA SILVA', 119)]


('SÃO GABRIEL DA CACHOEIRA-AM', 19, 92)
[('LUIZ INÁCIO LULA DA SILVA', 195)]


('TABATINGA-AM', 36, 100)
[('LUIZ INÁCIO LULA DA SILVA', 132)]


('SEABRA-BA', 88, 181)
[('LUIZ INÁCIO LULA DA SILVA', 141)]


('ICATU-MA', 31, 197)
[('LUIZ INÁCIO LULA DA SILVA', 44)]


('BACURI-MA', 107, 38)
[('LUIZ INÁCIO LULA DA SILVA', 106)]


('CORONEL JOSÉ DIAS-PI', 13, 242)
[('LUIZ INÁCIO LULA DA SILVA', 119)]


('SÃO DOMINGOS DO CAPIM-PA', 50, 181)
[('LUIZ INÁCIO LULA DA SILVA', 103)]


('SERRANO DO MARANHÃO-MA', 107, 156)
[('LUIZ INÁCIO LULA DA SILVA', 61)]


('GUARULHOS-SP', 185, 372)
[('LUIZ INÁCIO LULA DA SILVA', 10)]


('JANUÁRIA-MG', 148, 276)
[('LUIZ INÁCIO LULA DA SILVA', 115)]


('ITAPECURU MIRIM-MA', 16, 337)
[('LUIZ INÁCIO LULA DA SILVA', 89)]


('PRESIDENTE VARGAS-MA', 50, 151)
[('LUIZ INÁ

143

In [157]:
#mostrar urnas que lula teve zero votos
total_urnas_sem_lula = 0
for index, chave_votos in enumerate(dict_turno2_municipio_zona_secao):
    #print(dict_turno2_municipio_zona_secao[chave_votos])
    
    if len(dict_turno2_municipio_zona_secao[chave_votos]) < 4: #chance de não terem votado em L nem B
        achou_lula = False
        for t in dict_turno2_municipio_zona_secao[chave_votos]:
            if 'LUIZ INÁCIO LULA DA SILVA' in t:
                achou_lula = True
                
        if achou_lula == False:
            total_urnas_sem_lula += 1
            print(chave_votos)
            print(dict_turno2_municipio_zona_secao[chave_votos])
            print("\n")
total_urnas_sem_lula

('CHAVES-PA', 17, 41)
[('JAIR MESSIAS BOLSONARO', 39)]


('CHARRUA-RS', 100, 15)
[('JAIR MESSIAS BOLSONARO', 79)]


('CARACAS-ZZ', 1, 79)
[('JAIR MESSIAS BOLSONARO', 1)]


('CARACAS-ZZ', 1, 78)
[('JAIR MESSIAS BOLSONARO', 5)]




4

In [158]:
#total de votos pro lula nas urnas que ele teve 100%
soma_votos_lula_urnas100porcent

16455

In [160]:
#somar votos totais de lula e bolsonaro
votos_lula_total = 0
votos_bolsonaro_total = 0

for cidade in dict_cidades_2t:
    votos_lula_total += dict_cidades_2t[cidade]['LUIZ INÁCIO LULA DA SILVA']
    votos_bolsonaro_total += dict_cidades_2t[cidade]['JAIR MESSIAS BOLSONARO']

In [161]:
#resultado real da eleição
porcentos_lula = votos_lula_total / (votos_lula_total + votos_bolsonaro_total)
porcentos_bolso = votos_bolsonaro_total / (votos_lula_total + votos_bolsonaro_total)

votos_lula_total, votos_bolsonaro_total, porcentos_lula, porcentos_bolso

(60345999, 58206354, 0.5090240511717216, 0.49097594882827844)

In [162]:
#se transferir todos os votos de 100% das urnas do lula pro bolsonaro
votos_lula_total -= soma_votos_lula_urnas100porcent
votos_bolsonaro_total += soma_votos_lula_urnas100porcent

porcentos_lula = votos_lula_total / (votos_lula_total + votos_bolsonaro_total)
porcentos_bolso = votos_bolsonaro_total / (votos_lula_total + votos_bolsonaro_total)

votos_lula_total, votos_bolsonaro_total, porcentos_lula, porcentos_bolso

(60329544, 58222809, 0.5088852517334683, 0.4911147482665317)